# Importing Packages

In [38]:
import os
import sys
import pandas as pd
import nltk
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.tokenize import regexp_tokenize, word_tokenize, RegexpTokenizer
import matplotlib.pyplot as plt
import string
import re
import numpy as np
import itertools
from nltk import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import pos_tag
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import ConfusionMatrixDisplay, recall_score,\
    accuracy_score, precision_score, f1_score, classification_report
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('fake reviews dataset.csv')
df.head()

,category,rating,label,text_
0,Home_and_Kitchen_5,5.0,CG,"Love this! Well made, sturdy, and very comfor..."
1,Home_and_Kitchen_5,5.0,CG,"love it, a great upgrade from the original. I..."
2,Home_and_Kitchen_5,5.0,CG,This pillow saved my back. I love the look and...
3,Home_and_Kitchen_5,1.0,CG,"Missing information on how to use it, but it i..."
4,Home_and_Kitchen_5,5.0,CG,Very nice set. Good quality. We have had the s...


## Data Undersatding

In [3]:
df['category'].value_counts()

category
Kindle_Store_5                  4730
Books_5                         4370
Pet_Supplies_5                  4254
Home_and_Kitchen_5              4056
Electronics_5                   3988
Sports_and_Outdoors_5           3946
Tools_and_Home_Improvement_5    3858
Clothing_Shoes_and_Jewelry_5    3848
Toys_and_Games_5                3794
Movies_and_TV_5                 3588
Name: count, dtype: int64

In [4]:
df.describe()

,rating
count,40432.000000
mean,4.256579
std,1.144354
min,1.000000
25%,4.000000
50%,5.000000
75%,5.000000
max,5.000000


In [37]:
# Creating 'target' Column for Classification
df['target'] = np.where(df['label'] == 'CG', 1, 0)
df

,category,rating,label,text_,target,text_preproccesed
0,Home_and_Kitchen_5,5.0,CG,"Love this! Well made, sturdy, and very comfor...",1,love well make sturdy comfortable i love very ...
1,Home_and_Kitchen_5,5.0,CG,"love it, a great upgrade from the original. I...",1,love great upgrade original i mine couple year
2,Home_and_Kitchen_5,5.0,CG,This pillow saved my back. I love the look and...,1,this pillow save back i love look feel pillow
3,Home_and_Kitchen_5,1.0,CG,"Missing information on how to use it, but it i...",1,miss information use great product price i
4,Home_and_Kitchen_5,5.0,CG,Very nice set. Good quality. We have had the s...,1,very nice set good quality we set two month
...,...,...,...,...,...,...
40427,Clothing_Shoes_and_Jewelry_5,4.0,OR,I had read some reviews saying that this bra r...,0,i read review say bra run small i order two ba...
40428,Clothing_Shoes_and_Jewelry_5,5.0,CG,I wasn't sure exactly what it would be. It is ...,1,i sure exactly would it little large small siz...
40429,Clothing_Shoes_and_Jewelry_5,2.0,OR,"You can wear the hood by itself, wear it with ...",0,you wear hood wear hood wear jacket without ho...
40430,Clothing_Shoes_and_Jewelry_5,1.0,CG,I liked nothing about this dress. The only rea...,1,i like nothing dress the reason i give star i ...


## Text Preprocessing: Tokenization

In [7]:
# Tokenizing the text data in the 'text_' column of df
def tokenizer(x):
    
    corpus = [word_tokenize(doc) for doc in x]

# getting common stop words in english that we'll remove during tokenization/text normalization
    stop_words = stopwords.words('english')
    corpus_no_stopwords = []
    for words in corpus:
        docs = [x.lower() for x in words if ((x.isalpha()) & (x not in stop_words))]
        corpus_no_stopwords.append(docs)
    return corpus_no_stopwords

## Lemmantizer

In [39]:
# Defining new function 'lemmatizer'. This function takes two arguments: corpus(list of sentences or text data that we want to lemmatize), and as_string 

def lemmatizer(corpus, as_string=True):
    lem = WordNetLemmatizer()
# Defining an inner function 'pos_tagger'
    def pos_tagger(nltk_tag):
        if nltk_tag.startswith('J'):
            return wordnet.ADJ
        elif nltk_tag.startswith('V'):
            return wordnet.VERB
        elif nltk_tag.startswith('N'):
            return wordnet.NOUN
        elif nltk_tag.startswith('R'):
            return wordnet.ADV
        else:         
            return None
    lemmatized_corpus = []
    for sentence in corpus:
        pos_tags = pos_tag(sentence)
        lemmatized_sentence = []
        for word, tag in pos_tags:
            pos = pos_tagger(tag)
            if pos is not None:
                lemmatized_word = lem.lemmatize(word, pos)
            else:
                lemmatized_word = lem.lemmatize(word)
            lemmatized_sentence.append(lemmatized_word)
        lemmatized_corpus.append(lemmatized_sentence)
    if as_string:
        lemmatized_corpus  = [' '.join(x) for x in lemmatized_corpus]
    return lemmatized_corpus
# After processing all words in the sentence, the lemmatized_sentence is added to the lemmatized_corpus

In [9]:
# Tokenizing a text corpus
corpus_tokenized = tokenizer(df['text_'])

In [10]:
# Lemmatizing a text corpus
lemmatized_corpus = lemmatizer(corpus_tokenized)

## Pre vectorizing

In [11]:
# Adding Preprocessed Text Column to DataFrame
joined_lemm_corpus = [' '.join(x) for x in lemmatized_corpus]
df['text_preproccesed'] = pd.Series(data=lemmatized_corpus)

In [13]:
# Creating Document-Term Matrix Using CountVectorizer
vec = CountVectorizer(min_df = 0.05, max_df = 0.95)
X = vec.fit_transform(lemmatized_corpus)
countvec_df = pd.DataFrame(X.toarray(), columns=vec.get_feature_names_out())

In [17]:
# Creating TF-IDF Matrix Using TfidfVectorizer
tfidf = TfidfVectorizer(min_df = 0.05, max_df = 0.95)
Y = tfidf.fit_transform(lemmatized_corpus)
tfidf_df = pd.DataFrame(Y.toarray(), columns=tfidf.get_feature_names_out())

In [18]:
tfidf_df

,also,anyone,best,big,bit,book,buy,ca,character,come,...,try,two,use,want,way,well,work,would,write,year
0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.365431,0.000000,0.000000,0.0,0.000000
1,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.728227
2,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
3,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.436464,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
4,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.0,0.000000,...,0.000000,0.350925,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40427,0.000000,0.0,0.112657,0.000000,0.000000,0.0,0.00000,0.000000,0.0,0.103073,...,0.115232,0.206830,0.000000,0.099786,0.000000,0.075666,0.000000,0.077660,0.0,0.000000
40428,0.000000,0.0,0.000000,0.091199,0.000000,0.0,0.13296,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.061434,0.067497,0.252213,0.0,0.000000
40429,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.0,0.000000,...,0.174608,0.000000,0.000000,0.000000,0.000000,0.114655,0.000000,0.117676,0.0,0.000000
40430,0.064548,0.0,0.000000,0.000000,0.145785,0.0,0.11615,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.107335,0.000000,0.055082,0.0,0.000000


In [19]:
X

<40432x89 sparse matrix of type '<class 'numpy.int64'>'
	with 375538 stored elements in Compressed Sparse Row format>

### Splitting Data into Training and Testing Sets

In [20]:
X_train, X_test, y_train,y_test = train_test_split(df['text_'],df['target'], test_size=0.3,random_state=42)

In [21]:
# Preprocessing Text Data in Training and Testing Sets
X_train_preprocessed = lemmatizer(tokenizer(X_train))
X_test_preprocessed = lemmatizer(tokenizer(X_test))

### Random Forest and Logistic Regression with the Count Vectorizer

In [22]:
def scores(y_test, y_pred):
    print(f"""    Accuracy: {round(accuracy_score(y_test,y_pred),3)}
    
    Recall: {round(recall_score(y_test,y_pred),3)}
    
    F1: {round(f1_score(y_test,y_pred),3)}
    
    Precision: {round(precision_score(y_test,y_pred),3)}""")

In [23]:
steps = [('countvec',CountVectorizer(min_df = 0.05, max_df = 0.95)),('rfc',RandomForestClassifier(n_estimators=200))]
pipe_cv_rf = Pipeline(steps)
pipe_cv_rf.fit(X_train_preprocessed, y_train)
y_pred_cv_rf = pipe_cv_rf.predict(X_test_preprocessed)

In [24]:
steps[1] = ('logreg',LogisticRegression())
pipe_cv_lr = Pipeline(steps)
pipe_cv_lr.fit(X_train_preprocessed, y_train)
y_pred_cv_lr = pipe_cv_lr.predict(X_test_preprocessed)

### Random Forest and Logistic Regression with the Tfidf Vectorizer

In [25]:
steps = [('tfidfvec',TfidfVectorizer(min_df = 0.05, max_df = 0.95)),('rfc',RandomForestClassifier(n_estimators=200))]
pipe_idf_rf = Pipeline(steps)
pipe_idf_rf.fit(X_train_preprocessed, y_train)
y_pred_idf_rf = pipe_idf_rf.predict(X_test_preprocessed)

In [26]:
steps[1] = ('logreg',LogisticRegression())
pipe_idf_lr = Pipeline(steps)
pipe_idf_lr.fit(X_train_preprocessed, y_train)
y_pred_idf_lr = pipe_idf_lr.predict(X_test_preprocessed)

### Calculate Evaluation Scores for Random Forest Model

In [27]:
# Model 1: Random Forest with CountVectorizer
scores(y_test, y_pred_cv_rf)

    Accuracy: 0.766
    
    Recall: 0.754
    
    F1: 0.763
    
    Precision: 0.772


In [46]:
# Model 2: Random Forest with TF-IDF Vectorizer
scores(y_test,y_pred_idf_rf)

    Accuracy: 0.768
    
    Recall: 0.746
    
    F1: 0.763
    
    Precision: 0.781


### Calculate Evaluation Scores for Linear Regression Model

In [29]:
# Model 3: Logistic Regression with CountVectorizer
scores(y_test,y_pred_cv_lr)

    Accuracy: 0.72
    
    Recall: 0.716
    
    F1: 0.719
    
    Precision: 0.722


In [30]:
# Model 4: Logistic Regression with TF-IDF Vectorizer
scores(y_test,y_pred_idf_lr)

    Accuracy: 0.725
    
    Recall: 0.74
    
    F1: 0.729
    
    Precision: 0.718


- We evaluated four models, two using the CountVectorizer and two using the TF-IDF Vectorizer. Based on the evaluation scores, we decided to proceed with the Random Forest model using the TF-IDF Vectorizer as it achieved the highest accuracy and precision.

In [34]:
steps_rf = [('countvec',CountVectorizer(min_df = 0.05, max_df = 0.95)),('rfc',RandomForestClassifier(n_estimators=200,random_state=42))]
steps_lr = [('countvec',CountVectorizer(min_df = 0.05, max_df = 0.95)),('logreg',LogisticRegression(random_state = 42, max_iter=10000), )]

pipe_rf = Pipeline(steps_rf)
pipe_lr = Pipeline(steps_lr)

In [35]:
param_rf = {'rfc__n_estimators':[100,150,200],
                       'rfc__max_depth':[2,3,4,5],
                       'rfc__min_samples_leaf':[1,2,3]
                      }

param_lr = {'logreg__penalty':['l1', 'l2', 'elasticnet', None],
                 'logreg__C':[0.001,0.01,0.1,1,10,100],
                 'logreg__solver':['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga']
                }

In [36]:
gs_rf = GridSearchCV(estimator=pipe_rf, param_grid=param_rf,scoring='accuracy',cv=5,error_score=0)
gs_lr = GridSearchCV(estimator=pipe_lr, param_grid=param_lr,scoring='accuracy',cv=5,error_score=0)
gs_rf.fit(X_train_preprocessed,y_train)
gs_lr.fit(X_train_preprocessed,y_train)

/opt/homebrew/Caskroom/miniforge/base/envs/learn-env/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1193: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/learn-env/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1193: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/learn-env/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1193: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/learn-env/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1193: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/learn-env/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1193: UserWarning: Setting pe

GridSearchCV(cv=5, error_score=0,
             estimator=Pipeline(steps=[('countvec',
                                        CountVectorizer(max_df=0.95,
                                                        min_df=0.05)),
                                       ('logreg',
                                        LogisticRegression(max_iter=10000,
                                                           random_state=42))]),
             param_grid={'logreg__C': [0.001, 0.01, 0.1, 1, 10, 100],
                         'logreg__penalty': ['l1', 'l2', 'elasticnet', None],
                         'logreg__solver': ['lbfgs', 'liblinear', 'newton-cg',
                                            'newton-cholesky', 'sag', 'saga']},
             scoring='accuracy')

In [43]:
print(gs_rf.best_params_)
print(gs_rf.best_score_)

{'rfc__max_depth': 5, 'rfc__min_samples_leaf': 2, 'rfc__n_estimators': 100}
0.7035902146170281


In [44]:
print(gs_lr.best_params_)
print(gs_lr.best_score_)

{'logreg__C': 1, 'logreg__penalty': 'l1', 'logreg__solver': 'liblinear'}
0.7268742427732242
